#Step 1: Choose Your 10 Categories

In [1]:
selected_classes = [
    "pizza", "sushi", "ice_cream", "hamburger",
    "fried_rice", "ramen", "steak", "donuts",
    "pancakes", "spaghetti_bolognese"
]


#Step 2: Create a New Dataset (Train/Test)

In [3]:
import os, shutil

base_dir = "food-101"
image_dir = os.path.join(base_dir, "images")
meta_dir = os.path.join(base_dir, "meta")

train_txt = os.path.join(meta_dir, "train.txt")
test_txt = os.path.join(meta_dir, "test.txt")

new_train_dir = "food10-train"
new_test_dir = "food10-test"

os.makedirs(new_train_dir, exist_ok=True)
os.makedirs(new_test_dir, exist_ok=True)

def filter_split(txt_file, dest_dir, classes):
    with open(txt_file, 'r') as f:
        for line in f:
            path = line.strip() + ".jpg"
            class_name = path.split("/")[0]
            if class_name in classes:
                os.makedirs(os.path.join(dest_dir, class_name), exist_ok=True)
                shutil.copy(
                    os.path.join(image_dir, path),
                    os.path.join(dest_dir, path)
                )

filter_split(train_txt, new_train_dir, selected_classes)
filter_split(test_txt, new_test_dir, selected_classes)


#Step 3: Load Your New Dataset

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    rotation_range=15
)

train_generator = train_datagen.flow_from_directory(
    new_train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    new_train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    new_test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 6000 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


#Step 4: Train a Model (Fast MobileNetV2 Example)

In [5]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.3347 - loss: 2.0048 - val_accuracy: 0.7647 - val_loss: 0.8836
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 93s 497ms/step - accuracy: 0.6999 - loss: 0.9669 - val_accuracy: 0.7887 - val_loss: 0.7150
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 93s 494ms/step - accuracy: 0.7429 - loss: 0.7989 - val_accuracy: 0.7993 - val_loss: 0.6353
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 92s 492ms/step - accuracy: 0.7852 - loss: 0.6856 - val_accuracy: 0.8173 - val_loss: 0.5953
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 93s 494ms/step - accuracy: 0.8025 - loss: 0.6475 - val_accuracy: 0.8120 - val_loss: 0.5850
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 94s 503ms/step - accuracy: 0.8119 - loss: 0.5993 - val_accuracy: 0.8280 - val_loss: 0.5439
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 94s 498ms/step - accuracy: 0.8159 - loss: 0.5952 - val_accuracy: 0.8220 - val_loss: 0.5689
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 95s 506ms/step - accuracy: 0.8284 - loss: 0.5

#Step 5: Evaluate & Save

In [6]:
model.evaluate(test_generator)
model.save("food10_mobilenetv2.h5")

79/79 ━━━━━━━━━━━━━━━━━━━━ 54s 679ms/step - accuracy: 0.8756 - loss: 0.3588


# 6 Load Your Trained Model

In [7]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os

# Load model
model = load_model("food10_mobilenetv2.h5")  # Change name if needed

# Class names (must match your training order)
class_labels = [
    "pizza", "sushi", "ice_cream", "hamburger",
    "fried_rice", "ramen", "steak", "donuts",
    "pancakes", "spaghetti_bolognese"
]


# 7 Select an Image to Test

In [11]:
# Provide the path to your image
img_path = "Screenshot 2025-08-26 005935.png"  # Replace with your image path

# Load and preprocess
IMG_SIZE = (224, 224)
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # normalize


# 8 Make a Prediction

In [12]:
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)[0]
confidence = np.max(predictions) * 100

print(f"Predicted class: {class_labels[predicted_class]}")
print(f"Confidence: {confidence:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Predicted class: donuts
Confidence: 86.02%
